In [1]:
import unittest

In [2]:
def generate_sequence(n):
  import random

  bases = ['a','g','c','t']
  seq = ''
  for i in range(n):
    seq += bases[random.randint(0,3)]
  return seq

def chop_sequence(sequence,min_len=8,max_len=15,min_overlap=5):
  import random

  start = 0
  end = max(random.randint(start,max_len),min_len)
  chop = [sequence[start:end]]
  while end < len(sequence):
    start = random.randint(start + 1,end - min_overlap)
    end = max(random.randint(start,start + max_len),start + min_len)
    chop += [sequence[start:end]]
  return chop
  
ref = "Fresh out of college, Barry the Bee (Jerry Seinfeld) finds the prospect of working with honey uninspiring. He flies outside the hive for the first time and talks to a human (Renée Zellweger), breaking a cardinal rule of his species. Barry learns that humans have been stealing and eating honey for centuries, and he realizes that his true calling is to obtain justice for his kind by suing humanity for theft." #generate_sequence(200)
reads = chop_sequence(ref,9,18,8)
import random
# random.shuffle(reads)
print(ref)
print(reads)
# seq = Sequitur(reads)
# seq.construct()

Fresh out of college, Barry the Bee (Jerry Seinfeld) finds the prospect of working with honey uninspiring. He flies outside the hive for the first time and talks to a human (Renée Zellweger), breaking a cardinal rule of his species. Barry learns that humans have been stealing and eating honey for centuries, and he realizes that his true calling is to obtain justice for his kind by suing humanity for theft.
['Fresh out', 'resh out o', 'esh out of col', 't of coll', ' of colle', 'of colleg', 'f college, Barr', 'college, Barry ', 'ollege, Barry the ', ' Barry th', 'Barry the Bee', ' the Bee ', 'the Bee (', 'he Bee (J', 'e Bee (Jerry Sein', ' (Jerry Seinf', '(Jerry Sei', 'Jerry Sei', 'erry Sein', 'rry Seinf', 'ry Seinfe', 'y Seinfel', ' Seinfeld', 'Seinfeld) finds th', 'einfeld) finds th', ') finds the pr', 'nds the p', 'ds the pr', 's the prosp', 'he prospe', 'e prospec', ' prospect ', 'prospect ', 'rospect o', 'ospect of', 'spect of w', 'pect of w', 'ect of working', 'ct of working ', 't

In [1]:
class Node:
  def __init__(self,read,root):
    if root in read:
      self.read = read
      self.pre, self.root, self.suf = read.partition(root)
      self.pre = Prefix(node=self)
    else:
      raise Exception("Invalid root. Please select a root which exists in the read.")

In [7]:
# BUG: imperfect prefix split

class Prefix:
# class Appendix:
    def __init__(self,node=None,initus=None,terminus=None):
        import warnings

        self.node = node
        if type(initus) == int: self.initus = initus
        else: self.initus = 0
        if type(terminus) == int : self.terminus = terminus
        else: self.terminus = len(node.pre)
        if self.initus > self.terminus: warnings.warn("Prefix initus ahead of prefix initus. This will result in an empty prefix string.")
        self.slice = slice(self.initus,self.terminus)
        if len(node.pre) == 0 or terminus == 0: self.fix = ''
        else: self.fix = node.pre[self.slice]

    def __len__(self):
        return len(self.fix)
        
    def __repr__(self):
        if len(self.fix) == 0: return '^'
        return self.fix

    def __str__(self):
        if len(self.fix) == 0: return '^'
        return self.fix
    
    def __hash__(self):
        if len(self.fix) > 0: return hash(self.fix[-1])
        return hash('')

    def __eq__(self,other):
        if type(other) == str:
            if len(self.fix) > 0 and len(other) > 0: return self.fix[-1] == other[-1]
            return len(self.fix) == len(other) == 0
        else:
            if len(self.fix) > 0 and len(other.fix) > 0: return self.fix[-1] == other.fix[-1]
            return len(self.fix) == len(other.fix) == 0
    
    def __sub__(self,other):
        self.fix = self.fix[:self.fix.rindex(str(other.fix))]
        return self
    
    def __getitem__(self,index):
        return self.fix[index]
    
    def __contains__(self,other):
        return (self.fix.endswith(other.fix) or other.fix.endswith(self.fix)) and min(len(self.fix),len(other.fix)) >= len(self.node.root)

    # def __enter__(self):
        

    def match(self,other):
        i = 0
        while self.fix[-1-i:] == other.fix[-1-i:] and min(len(self.fix),len(other.fix)) >= i: i+=1
        # if i == min(len(self.fix),len(other.fix)) > len(self.node.root) and self.fix[-1-i:] == other.fix[-1-i:]: #TODO: case: if one prefix contains a significant portion of another
        #     self.suf.add_variant(other.suf)
        s_pre = Prefix(node=self.node,terminus=self.terminus-i)
        o_pre = Prefix(node=other.node,terminus=other.terminus-i)
        
        self.initus = self.terminus - i
        self.slice = slice(self.initus,self.terminus)
        self.fix = self.node.pre[self.slice]
        return s_pre, o_pre
        

In [3]:
class SequiterTest(unittest.TestCase):
    # node tests
    def test_node_init(self):
        # case 1: root is initus
        n = Node("abcdefghij","abc")
        self.assertEqual(n.read,"abcdefghij")
        self.assertEqual(n.pre,"")
        self.assertEqual(n.root,"abc")
        self.assertEqual(n.suf,"defghij")
        self.assertEqual(type(n.pre),Prefix)
        # case 2: root is middling
        n = Node("abcdefghij","efg")
        self.assertEqual(n.read,"abcdefghij")
        self.assertEqual(n.pre,"abcd")
        self.assertEqual(n.root,"efg")
        self.assertEqual(n.suf,"hij")
        self.assertEqual(type(n.pre),Prefix)
        # case 3: root is terminus
        n = Node("abcdefghij","hij")
        self.assertEqual(n.read,"abcdefghij")
        self.assertEqual(n.pre,"abcdefg")
        self.assertEqual(n.root,"hij")
        self.assertEqual(n.suf,"")
        self.assertEqual(type(n.pre),Prefix)
        # case 4: root is length of read
        n = Node("abcdefghij","abcdefghij")
        self.assertEqual(n.read,"abcdefghij")
        self.assertEqual(n.pre,"")
        self.assertEqual(n.root,"abcdefghij")
        self.assertEqual(n.suf,"")
        self.assertEqual(type(n.pre),Prefix)
        # case 5: root is not in read
        self.assertRaises(Exception,Node,"abcdefghij","xyz")
    
    # prefix tests
    def test_prefix_init(self):
        # case 1: no terminus, no initus, initus root
        node = Node("abcdefghij","abc")
        p = Prefix(node)
        self.assertEqual(p.node,node)
        self.assertEqual(p.initus,0)
        self.assertEqual(p.terminus,0)
        self.assertEqual(p.slice,slice(0,0))
        self.assertEqual(p.fix,"")
        # case 2: no terminus, no initus, middling root
        node = Node("abcdefghij","def")
        p = Prefix(node)
        self.assertEqual(p.node,node)
        self.assertEqual(p.initus,0)
        self.assertEqual(p.terminus,len("def"))
        self.assertEqual(p.slice,slice(0,len("def")))
        self.assertEqual(p.fix,"abc")
        # case 3: no terminus, no initus, terminus root
        node = Node("abcdefghij","hij")
        p = Prefix(node)
        self.assertEqual(p.node,node)
        self.assertEqual(p.initus,0)
        self.assertEqual(p.terminus,len("abcdefg"))
        self.assertEqual(p.slice,slice(0,len("abcdefg")))
        self.assertEqual(p.fix,"abcdefg")
        # case 4: setting initus, not setting terminus
        node = Node("abcdefghij","hij")
        p = Prefix(node,3)
        self.assertEqual(p.node,node)
        self.assertEqual(p.initus,3)
        self.assertEqual(p.terminus,len("abcdefg"))
        self.assertEqual(p.slice,slice(3,len("abcdefg")))
        self.assertEqual(p.fix,"defg")
        # case 5: not setting initus, setting terminus
        node = Node("abcdefghij","hij")
        p = Prefix(node,None,6)
        self.assertEqual(p.node,node)
        self.assertEqual(p.initus,0)
        self.assertEqual(p.terminus,6)
        self.assertEqual(p.slice,slice(0,6))
        self.assertEqual(p.fix,"abcdef")
        # case 6: setting initus > terminus
        node = Node("abcdefghij","fgh")
        with self.assertWarns(UserWarning,Prefix,node,3,1,) as w:
            print(w)
        # p = Prefix(node,3,1)
        # self.assertEqual(p.node,node)
        # self.assertEqual(p.initus,3)
        # self.assertEqual(p.terminus,1)
        # self.assertEqual(p.slice,slice(3,1))
        # self.assertEqual(p.fix,"")
    # # transition table tests
    # def test_transition_table_init(self):
    #     t = TransitionsTable()
    #     self.assertTrue(type(t.transitions) == dict)

    # def test_transition_table_add(self):
    #     t = TransitionsTable()
    #     items = [Node("abcdefghij","abc"),Node("abcdefghij","abc"),Node("abcdefghij","bcd"),Node("abcdhijefg","abc")]
    #     # case 1: empty table
    #     self.assertTrue(len(t.transitions) == 0)
    #     t.add(items[0])
    #     self.assertTrue(len(t.transitions) == 1)
    #     self.assertTrue("abc" in t.transitions.keys())
    #     self.assertTrue(type(t.transitions["abc"]) == Branch)
    #     # case 2: non-empty table
    #     # case 2.1: node with same read and same root
    #     self.assertTrue(len(t.transitions) == 1)
    #     t.add(items[1])
    #     self.assertTrue(len(t.transitions) == 1)
    #     self.assertTrue("abc" in t.transitions.keys())
    #     self.assertTrue(type(t.transitions["abc"]) == Branch)
    #     # case 2.2: node with same read and different root
    #     self.assertTrue(len(t.transitions) == 1)
    #     t.add(items[2])
    #     self.assertTrue(len(t.transitions) == 2)
    #     self.assertTrue("bcd" in t.transitions.keys())
    #     self.assertTrue(type(t.transitions["abc"]) == Branch and type(t.transitions["bcd"]) == Branch)
    #     # case 2.3: node with different read and same root
    #     self.assertTrue(len(t.transitions["abc"].transitions) == 1)
    #     print(t.transitions["abc"])
    #     t.add(items[3])
    #     print(t.transitions["abc"])
    #     self.assertTrue(len(t.transitions["abc"].transitions) == 2)
    #     self.assertTrue(type(t.transitions["abc"]) == Branch)

unittest.main(argv=[''],exit=False,verbosity=2)

NameError: name 'unittest' is not defined

In [8]:
class TransitionsTable:
    def __init__(self):
        self.transitions = {}

    def add(self,item):
        if item.root in self.transitions: self.transitions[item.root].add(item)
        else: self.transitions[item.root] = Branch(self,item.root,item)

    def values(self):
        return self.transitions.values()
    
    def pop(self,key):
        if key in self.transitions: self.transitions.pop(key)
        
    def __repr__(self):        
        return str(self.transitions)
        
    def __setitem__(self,key,value):
        self.transitions[key] = value

    def __getitem__(self,key):
        return self.transitions[key]

In [9]:
 # BUG: how to handle insignificant prefixes
 # BUG: branching with no common prefix

class Branch(TransitionsTable):
    def __init__(self,node,super_key,item=None):
        super().__init__()
        self.node = node
        self.super_key = super_key
        self.items = {}
        if item: self.add(item)
    
    def add(self,item):
        if item.pre in self:
            if type(self.transitions[str(item.pre)]) == Branch:
                self.transitions[str(item.pre)].add(item)
                return
            n = self.items[str(item.pre)]
            branch = Branch(self,n.pre) # BUG: replaces branch but should add to it
            self.transitions[str(item.pre)] = branch
            n.pre, item.pre = n.pre.match(item.pre)
            branch.add(n)
            branch.add(item)
        else:
            self.transitions[str(item.pre)] = item.suf
            # self.items[str(item.pre)] = item

    def __contains__(self,key):
        # if type(key) == str: 
        #     for n in self.transitions.keys():
        #         print(key in n)
        return key in self.transitions
        
    def __repr__(self):
        return str(self.transitions)

    # def __len__(self):
    #     return len()

    # def __getitem__(self, key):
        

In [10]:
class AttenuatedPrefix(Prefix):
    def __init__(self,prefix):
        self.fix = prefix

    def __len__(self):
        return len(self.fix)
    

In [11]:
# NOTE: k must be less than the shortest overlap
# TODO: actual assembly

tr = TransitionsTable()
k = 3
for read in ['you say hel',
                ' say hello wo',
                        'lo world, i be',#'ld, i bellow go t',
                              'ld, i bellow go t',
                                        'ow go to hell']:
    for i in range(len(read)-k+1):
        tr.add(Node(read,read[i:i+k]))

In [12]:
tr['ell']

{' say h': 'o wo', 'ld, i b': 'ow go t', 'ow go to h': ''}

In [17]:
tr.transitions

{'you': {'^': ' say hel'},
 'ou ': {'y': 'say hel'},
 'u s': {'yo': 'ay hel'},
 ' sa': {'you': 'y hel', '^': 'y hello wo'},
 'say': {'you ': ' hel', ' ': ' hello wo'},
 'ay ': {'you s': 'hel', ' s': 'hello wo'},
 'y h': {'you sa': 'el', ' sa': 'ello wo'},
 ' he': {'you say': 'l', ' say': 'llo wo', 'ow go to': 'll'},
 'hel': {'you say ': '', ' say ': 'lo wo', 'ow go to ': 'l'},
 'ell': {' say h': 'o wo', 'ld, i b': 'ow go t', 'ow go to h': ''},
 'llo': {' say he': ' wo', 'ld, i be': 'w go t'},
 'lo ': {' say hel': 'wo', '^': 'world, i be'},
 'o w': {' say hell': 'o', 'l': 'orld, i be'},
 ' wo': {' say hello': '', 'lo': 'rld, i be'},
 'wor': {'lo ': 'ld, i be'},
 'orl': {'lo w': 'd, i be'},
 'rld': {'lo wo': ', i be'},
 'ld,': {'lo wor': ' i be', '^': ' i bellow go t'},
 'd, ': {'lo worl': 'i be', 'l': 'i bellow go t'},
 ', i': {'lo world': ' be', 'ld': ' bellow go t'},
 ' i ': {'lo world,': 'be', 'ld,': 'bellow go t'},
 'i b': {'lo world, ': 'e', 'ld, ': 'ellow go t'},
 ' be': {'lo worl

In [ ]:
tr = TransitionsTable()
k = 3
d = chop_sequence(generate_sequence(50))
for read in d:
    for i in range(len(read)-k+1):
        tr.add(Node(read,read[i:i+k]))

In [34]:
proto = 'lo world, i be'
b = tr.transitions[proto[-k:]]
att_pre = AttenuatedPrefix(proto[:-k])
while 1:
        while att_pre in b and type(b[att_pre]) == Branch:
                b = b[att_pre]
                att_pre -= b.super_key
        if att_pre - att_pre in b: 
                proto += b[att_pre - att_pre]
                b = tr.transitions[proto[-k:]]
                att_pre = AttenuatedPrefix(proto[:-k])
        else: break
proto

'lo world, i bellow go to hell'

In [18]:
d = chop_sequence(generate_sequence(50))
r = set()
k=3
for w in d:
    for i in range(len(w)-k+1):
        r.add(w[i:i+k])
r = list(r)
r.sort()

In [11]:
proto = 'Fresh out'
b = tr.transitions[proto[-k:]]
att_pre = AttenuatedPrefix(proto[:-k])
while 1:
        while att_pre in b and type(b[att_pre]) == Branch:
                b = b[att_pre]
                att_pre -= b.super_key
        if att_pre - att_pre in b: 
                proto += b[att_pre - att_pre]
                b = tr.transitions[proto[-k:]]
                att_pre = AttenuatedPrefix(proto[:-k])
        else: break
proto

'Fresh out'

In [9]:
proto = 'Fresh out'
b = tr.transitions[proto[-k:]]
att_pre = AttenuatedPrefix(proto[:-k])
# if att_pre in b and type(b[att_pre]) == Branch:
#         b = b[att_pre]
# #         att_pre -= b.super_key
# att_pre in b
b

{resh_: {F: '', : ' o'}}

In [56]:
# initalisation
seed = 'leg'
proto = transitions[seed]

while proto.suffix:
    proto = transitions[proto.suffix.root]
    seed += proto.root[-1]

In [36]:
s = 'she_sells_sea_sells_on_the_sea_shore'
r = set()
for i in range(len(s)-2):
    r.add(s[i:i+3])
print(len(r))

22


In [57]:
seed

'leger)'

In [14]:
import random

# initalisation
seed = 'leg'
proto = random.choice(transitions[seed])
extensions = [proto]
candidates = list(set(transitions[proto.root]).difference(extensions))

# build
while len(candidates) > 0:
    p = proto
    overlap_string = p.root
    new_terminus = False
    # # select an extender
    # # longest overlap wins
    for c in candidates:
        if overlap_string not in c.read: continue
        if c.read in p.read:
            overlap_string = c.read
            p = p.contig.kmers_tuple[p.read.index(overlap_string)]
            extensions += [c]
            continue
        if p.read in c.read:
            extensions += [c]
            proto = c.contig[-1]
            if proto.root ==  p.root: 
                p = proto.contig.kmers_tuple[proto.read.index(overlap_string)]
                overlap_string = p.read
                continue
            else: 
                new_terminus = True
                break

        c = c.contig.kmers_tuple[c.read.index(overlap_string)]
        while p.prefix and c.prefix and p.prefix.root == c.prefix.root:
            overlap_string = c.prefix.root[0] + overlap_string
            c = c.prefix
            p = p.prefix
        if p.prefix and c.prefix: continue # then a difference at that position

        # # change direction
        # c = c.contig.kmers_tuple[c.read.index(overlap_string)]
        # while p.prefix and c.prefix and p.prefix.root == c.prefix.root:
        #     overlap_string = c.prefix.root[0] + overlap_string
        #     c = c.prefix
        #     p = p.prefix
        # if p.prefix and c.prefix: continu#e

        extender = c
        # p = proto
    # # extend
    if new_terminus: continue
    proto += extender
    # while proto.suffix:
    #     proto = proto.suffix
    #     seed += proto.root[-1]
    extensions += [extender]
    candidates = list(set(transitions[proto.root]).difference(extensions))

KeyboardInterrupt: 

In [28]:
extensions

[ut of college, Ba,  out of colleg, ut of college, Ba]

In [262]:
list(set(transitions[proto.root]).difference())


[ of college, B, college, Barry t]

In [261]:
extensions

[college, Barry t,  college,,  of college, B]

In [87]:
seed

' of college, Barry the Bee (Jerry Seinfeld) finds the prospect of college, Barry the Bee (Jerry Seinfeld) finds the prospect of college, Barry the Bee (Jerry Seinfeld) finds the prospect of college, Barry the Bee (Jerry Seinfeld) finds the prospect of college, Barry the Bee (Jerry Seinfeld) finds the prospect of college, Barry the Bee (Jerry Seinfeld) finds the prospect of college, Barry the Bee (Jerr'

In [83]:
transitions

{'ct ': [ct of wor,
  the prospect of ,
  rospect o,
  spect of working ,
  ospect of,
  pect of wor,
   the prospect of ],
 't o': [ct of wor,
  esh out o,
  the prospect of ,
  rospect o,
  h out of colle,
  t of working wi,
  ut of col,
  spect of working ,
  ospect of,
  t of coll,
  pect of wor,
   the prospect of ,
  sh out of],
 ' of': [ct of wor,
  the prospect of ,
   of worki,
   of college, B,
  h out of colle,
  rdinal rule of h,
   of his s,
  t of working wi,
  e of his ,
  ut of col,
  l rule of his spe,
  spect of working ,
  ospect of,
  t of coll,
  pect of wor,
   the prospect of ,
  sh out of],
 'of ': [ct of wor,
  the prospect of ,
   of worki,
   of college, B,
  h out of colle,
  of his specie,
  rdinal rule of h,
   of his s,
  t of working wi,
  e of his ,
  ut of col,
  l rule of his spe,
  spect of working ,
  t of coll,
  pect of wor,
   the prospect of ,
  of working],
 'f w': [ct of wor,
   of worki,
  t of working wi,
  spect of working ,
  pect of wor,


In [5]:
kmer_to_read['wor']

['ct of wor',
 ' of worki',
 't of working wi',
 'spect of working ',
 'pect of wor',
 ' working ',
 'of working',
 'working w']

In [6]:
kmer_to_read['ng ']
        

['breaking a',
 'ng and eat',
 'ng humani',
 'uing humanity',
 'calling is to',
 '), breaking a ',
 'king a cardina',
 ' calling is ',
 'g and eating honey',
 't of working wi',
 'stealing ',
 'ng honey ',
 'aling and eati',
 'ng a cardinal rul',
 'rking with honey ',
 'n stealing and',
 'ing and e',
 'orking wi',
 'ng is to ob',
 ' by suing huma',
 'spect of working ',
 'ing human',
 'nd eating honey ',
 'eaking a ',
 'tealing a',
 'aking a c',
 ' working ',
 'ealing an',
 ' stealing an',
 'working w']